<a href="https://colab.research.google.com/github/TillVollmer5/mass_spectroscopy/blob/main/Concentration_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Concentration calculation
This notebook is used to calculate the concentration of phtalates in an environemental sample using an internal standard for quantitation. The Calibration_calculation.ipynb notebook, yields severall of the required files nescessary for the calculations in this notebook. The files *Cal_{n}_analyte_linreg.csv* need to be used which is located in the *calibration_output* folder on the google drive. If this notebook is not applied from my machine in connection with my google drive, either the path to the required files needs to be adabted or changed.
The following code allows for the calculation of the concentration, by using the linear regression form a calibration series and the ratio of the signal of the analyte with the internal standard (area). All the data regarding the calibration are present on my google drive in the folders: *calibration_input* and *calibration_output*, while a new google drive folder will be used to save the final data in as well as a new input folder.

*Google drive folder:*
quantitation_input
quantitation_output

The .csv files containing the peak list from the sample should be saved in the in the input folder. The names should follow a consequent naming sceme:

***Sample_{date_of_analysis}_{tag_bonus_info}_{Sample_Nr}_pl.csv***

Here the Sample number *(Sample_Nr)* is of key importance and needs to be properly documented in the Labjournal, as it is the main way of differentiation.


The following code is used to import all the nescessary libraries and mount the google drive.

In [1]:
import numpy as np
from google.colab import drive
import glob
import pandas as pd
import os
import csv
from datetime import datetime
import re
import ast

drive.mount('/content/drive', force_remount = True)

sample_nr_low = 184 #needs to be adapted to 1 nr in the series
sample_nr_high = 185 #needs to be adapted to the last nr of the series

Mounted at /content/drive


The following code is the addapted code from the Calibration_calculation.ipynb and yields the separated in separated .csv files, which will be used to calculate the ratio in a following code segment.

The output file has the following filename and is saved in the quantitation_output folder:

***Sample_{n}_row{i+1}.csv***

In [2]:
#Loop over all the designated sample files.
for n in range(sample_nr_low, sample_nr_high):
    files = glob.glob(f'/content/drive/My Drive/quantitation_input/*_{n}_pl.csv')

    if not files:
        print(f'Files matching pattern Sample Nr. {n} was not found.')
        continue

    else:
        # Loop over each row and save a separate data frame for each
        df = pd.read_csv(files[0], header=4)
        num_rows = len(df)

        for i in range(num_rows):
            row_dfs = []
            for file in files:
                df = pd.read_csv(file, header=4, na_values='')

                # Extract the i'th row of the CSV file and append it to the row data frame list
                row = df.iloc[i, :].dropna().astype(str)
                row_dfs.append(row)

            # Concatenate the row data frames along the columns axis and convert to a data frame
            combined_df = pd.concat(row_dfs, axis=1).T

            # Save the combined data frame to a CSV file with a name indicating the row number
            combined_df.to_csv(f'/content/drive/My Drive/quantitation_output/Sample_{n}_row{i+1}.csv', index=False, header=False)

            # Convert all values to strings before joining and print the row in the desired format
            formatted_row = '\t'.join(map(str, combined_df.values[0]))
            print(formatted_row)

13.81	13.76	13.88	2607633.81	100.0	1235924.703	100.0
15.74	15.7	15.78	21717329.846	7.79	9093567.441	7.39
18.59	18.54	18.64	4430178435.267	100.0	2110946770.352	100.0
20.6	20.54	20.65	55273574.35	19.83	26837409.11	21.82
25.6	25.58	25.61	2792083.987	1.0	1650299.516	1.34
26.74	26.68	26.88	198173303.103	71.08	84011444.595	68.31
29.32	29.3	29.32	847190.335	0.3	1385989.224	1.13


The following code allows to calculate the ratio of the analytes with the internal standard. The output is saved as:

***Sample_{n}_row*_ratio.csv**

In [3]:
# Loop over all the designated sample files. Currently works but i am not sure if it might be just exactly the same as the first one, it works but not iterative...
for n in range(sample_nr_low, sample_nr_high):
    divisor_file = f'/content/drive/My Drive/quantitation_output/Sample_{n}_row3.csv'
    directory = '/content/drive/My Drive/quantitation_output'

    if not os.path.isfile(divisor_file):
        print(f"Divisor file does not exist: {divisor_file}")
        continue  # Skip to the next iteration if the divisor file is missing

    divisor_df = pd.read_csv(divisor_file, sep=',', header=None)
    csv_files = glob.glob(os.path.join(directory, f'*{n}_row*.csv'))

    # Process each CSV file
    for csv_file in csv_files:
        if csv_file == divisor_file or '_ratio' in csv_file:
            continue  # Skip the divisor file itself and ratio files
        data_df = pd.read_csv(csv_file, sep=',', header=None)
        data_df = data_df.apply(pd.to_numeric, errors='coerce')  # Convert values to numeric
        print('divisor_df:', divisor_df)
        print('data_df:', data_df)
        divided_data_df = pd.DataFrame()

        for i in range(len(data_df)):
          # Perform division on the data on the complete dataframe
          divided_row = data_df.iloc[i] / divisor_df.iloc[i]
          divided_data_df = divided_data_df.append(divided_row, ignore_index=True)

        # Create a new file name with the "_ratio" tag
        new_file_name = os.path.splitext(csv_file)[0] + '_ratio.csv'

        divided_data_df.to_csv(new_file_name, sep=',', header=False, index=False)
        control_import_df = pd.read_csv(new_file_name, sep=',', header=None)

        # Compare the recalculated values with the content of the ratio file
        """if control_import_df.equals(divided_data_df):
            print(f"The files match: {new_file_name}")
        else:
            print(f"The files do not match: {new_file_name}")"""

        #print(control_import_df)
        print(divided_data_df)
        print(csv_file)
        print('****************************************************')
    print('===========================================================')

divisor_df:        0      1      2             3      4             5      6
0  18.59  18.54  18.64  4.430178e+09  100.0  2.110947e+09  100.0
data_df:        0      1      2           3      4            5      6
0  13.81  13.76  13.88  2607633.81  100.0  1235924.703  100.0
          0         1         2         3    4         5    6
0  0.742873  0.742179  0.744635  0.000589  1.0  0.000585  1.0
/content/drive/My Drive/quantitation_output/Sample_184_row1.csv
****************************************************
divisor_df:        0      1      2             3      4             5      6
0  18.59  18.54  18.64  4.430178e+09  100.0  2.110947e+09  100.0
data_df:        0     1      2             3     4            5     6
0  15.74  15.7  15.78  2.171733e+07  7.79  9093567.441  7.39
          0         1         2         3       4         5       6
0  0.846692  0.846818  0.846567  0.004902  0.0779  0.004308  0.0739
/content/drive/My Drive/quantitation_output/Sample_184_row2.csv
***********

<ipython-input-3-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-3-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-3-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-3-61a18a55ae68>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  divided_data_df = divided_data_df.append(divided_row, ignore_index=True)
<ipython-input-3-61a18a55ae6

          0         1         2         3      4         5       6
0  1.577192  1.580367  1.572961  0.000191  0.003  0.000657  0.0113
/content/drive/My Drive/quantitation_output/Sample_184_row7.csv
****************************************************


The following code takes the ratio files as well as the parameters for the linear regression (analyte specific), and yields a final output file that contains all the information, such as the information of the ratio used for the quantitaiton, the parameters for the linear equation as well as other information such as compound id and other general sample information.

The final outpur files have the following name format:

***Sample_{n}_quantitation_output.csv***


In [4]:
current_date = datetime.today().strftime('%Y-%m-%d')
output_dir = '/content/drive/My Drive/quantitation_export/'
#header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]
header_row2 = ['Analyte', 'Intercept', 'Slope', 'Value', 'Concentration [microg/ml]']

row_descriptors = ['place holder','Dimethylphthalate', 'Diethylphthalate', 'Internal standard', 'Dibutylphthalate', 'Benzylbutylphthalate', 'Bis(2-ethyl-hexyl)phthalate', 'Di-n-octylphthalate']
file_pattern = '/content/drive/My Drive/quantitation_output/Sample_{}_row{}_ratio.csv'

for n in range(sample_nr_low, sample_nr_high):
    output_file = output_dir + f'Sample_{n}_concentration_output_{current_date}.csv'
    header_row1 = [f'Analysis Nr. {n}', 'Phthalate concentration calculation', 'Concentration_calibration.ipynb', 'Till Vollmer', current_date]

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header_row1)
        writer.writerow(header_row2)

        for i in range(1, 8):
            if i == 3:
                continue
            print(f'Sample nr: {n}')
            print(f'Compound iteration: {i}')
            linreg_df = pd.read_csv(f'/content/drive/My Drive/calibration_export/Cal_{i}_analyte_linreg.csv')
            print(f'linear function information: {linreg_df}')
            descriptor = row_descriptors[i]
            print(f'Descriptor: {descriptor}')
            file_path = file_pattern.format(n, i)
            if not os.path.isfile(file_path):
                print(f"File not found for Sample {n}, Row {i}")
                continue

            sample_df = pd.read_csv(file_path, header=None, delimiter=',')
            print(f'Sample dataframe: {sample_df}')

            # Extracting the slope value from the linear function information
            intercept_str = str(linreg_df['Intercept'].values[0])
            slope_str = linreg_df['Slope'].values[0]

            # Remove the brackets before using regular expressions
            slope_str_cleaned = slope_str.strip('[]')
            slope_value = ast.literal_eval(slope_str_cleaned)
            if isinstance(slope_value, list):
                slope_value = slope_value[0]  # If slope is a list, take the first element

            intercept = float(re.findall(r'[-\d.]+', intercept_str)[0])
            slope = float(slope_value)

            fourth_column = sample_df.iloc[:, 3]  # Accessing the 4th column
            value = float(fourth_column.iloc[0])  # Assuming you want the value from the first row (index 0)
            concentration = slope * value + intercept
            print(f"Calculated concentration: {concentration}")
            row_data = [descriptor, intercept, slope, value, concentration]
            writer.writerow(row_data)

            # Additional print statements for debugging
            print(f"Sample DataFrame shape: {sample_df.shape}")
            print(f"Linreg DataFrame shape: {linreg_df.shape if linreg_df is not None else None}")
            print("------------------------------------------------------------------------")



Sample nr: 184
Compound iteration: 1
linear function information:    Intercept         Slope  R2 Value
0   0.001358  [0.35667912]  0.999713
Descriptor: Dimethylphthalate
Sample dataframe:           0         1         2         3    4         5    6
0  0.742873  0.742179  0.744635  0.000589  1.0  0.000585  1.0
Calculated concentration: 0.001567874924265275
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
Sample nr: 184
Compound iteration: 2
linear function information:    Intercept         Slope  R2 Value
0   0.001323  [0.38786679]  0.999676
Descriptor: Diethylphthalate
Sample dataframe:           0         1         2         3       4         5       6
0  0.846692  0.846818  0.846567  0.004902  0.0779  0.004308  0.0739
Calculated concentration: 0.003224615186556405
Sample DataFrame shape: (1, 7)
Linreg DataFrame shape: (1, 3)
------------------------------------------------------------------------
S